In [46]:
import pandas as pd
from pandas.
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.proportion import proportions_ztest

from scipy.stats import ttest_ind

import matplotlib.pyplot as plt
import seaborn as sns




## ANOVA (as a function of mass)

In [47]:
df = pd.read_csv("CSV_MASTERS/population.csv")
df.head()

,name,id,nametype,recclass,mass,fall,year,reclat,reclong,GeoLocation,major_classification,subclass_category,country,country_abrv,density_km_squared,density_mi_squared
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333,"(50.775, 6.08333)",Chondrite,L,Germany,DE,233.0,90.0
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333,"(56.18333, 10.23333)",Chondrite,H,Denmark,DK,133.0,52.0
2,Abee,6,Valid,EH4,107000.0,Fell,1952.0,54.21667,-113.00000,"(54.21667, -113.0)",Chondrite,E,Canada,CA,4.0,1.0
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000,"(16.88333, -99.9)",Achondrite,Acapulcoite,Mexico,MX,64.0,25.0
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000,"(-33.16667, -64.95)",Chondrite,L,Argentina,AR,16.0,6.0


In [48]:
formula = 'mass ~ C(country) + C(major_classification) + density_km_squared + reclat + reclong'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

                               sum_sq       df          F        PR(>F)
C(country)               1.290254e+15    121.0  25.976511  0.000000e+00
C(major_classification)  1.573244e+14      4.0  95.813538  3.449987e-81
density_km_squared       3.958162e+10      1.0   0.096424  7.561661e-01
reclat                   4.140434e+10      1.0   0.100864  7.507971e-01
reclong                  1.850500e+11      1.0   0.450796  5.019619e-01
Residual                 1.351435e+16  32922.0        NaN           NaN


## T Test comparsion of major_classifications

In [65]:
def test_classifications(classification_names=("Chondrite", "Achondrite", "Iron", "Stony-Iron")):
    """
    Accepts the names of the classifications (or a subset of classifications)
    
    For each classification, runs a t test against all others to determine if their masses are from the same population
    
    Returns a dataframe with the results. 
    (For the aplha columns, True and False refers to the rejection of the null hypothesis) 
    """
    
    
    classification_mass = []
    # append a list of masses to the classification_mass list for each name
    for name in classification_names:
        classification_mass.append(df[df.major_classification == name].mass)
    
    test_results = [] # where we will be appending all our results
    alpha_list = (0.1, 0.05, 0.01) # the different alpha levels to test
    for i in range(len(classification_mass)):
        for j in range(i + 1, len(classification_mass)):
            samp1, samp2 = (classification_names[i],classification_names[j]) 
            # test each classification against the others. 
            t_stat, p_value = ttest_ind(classification_mass[i], classification_mass[j], equal_var=False)
            # given the p values, check against list of alphas
            alpha_1, alpha_05, alpha_01 = (p_value < alpha_list[0],p_value < alpha_list[1], p_value < alpha_list[2])
            #  append everything to the rest results as a tuple
            test_results.append((samp1, samp2, t_stat, p_value, alpha_1, alpha_05, alpha_01))
    # create and return a dataframe of the results        
    return pd.DataFrame(test_results, columns=("sample_1", "sample_2", "t_stat", "p_value", "alpha 10%", "alpha 5%", "alpha 1%"))

In [66]:

# test_classifications().to_csv("tables/classification_test.csv", index=False)
test_classifications()

,sample_1,sample_2,t_stat,p_value,alpha 10%,alpha 5%,alpha 1%
0,Chondrite,Achondrite,-1.509633,0.131428,False,False,False
1,Chondrite,Iron,-4.238587,0.000025,True,True,True
2,Chondrite,Stony-Iron,-3.092844,0.002233,True,True,True
3,Achondrite,Iron,-4.187946,0.000031,True,True,True
4,Achondrite,Stony-Iron,-2.851040,0.004747,True,True,True
5,Iron,Stony-Iron,3.448491,0.000586,True,True,True


## T Test Comparison of Continents to Number Meteorites Observed (done in kate's branch)

Null Hypothesis = For any comparison, the number of meteorites comes from the same population

## T Test Comparison of country populations to number meteorites observed

In [51]:
country_names = list(df.country.value_counts().index)
hit_counts = list(df.country.value_counts())

# pd.DataFrame(list(zip(country_names, hit_counts)), columns=["country", "num_strikes"]).to_csv(
#     "CSV_MASTERS/strikes_per_country.csv", index=False
# )

## Strikes Per Global Quadrant

In [68]:
def test_quadrants(df):
    ne = df[(df.reclat >= 0) & (df.reclong >= 0)].mass
    nw = df[(df.reclat >= 0) & (df.reclong <= 0)].mass
    sw = df[(df.reclat <= 0) & (df.reclong <= 0)].mass
    se = df[(df.reclat <= 0) & (df.reclong >= 0)].mass 
    
    quadrant_data = (ne, nw, sw, se)
    quadrant_names = ("NE", "NW", "SW", "SE")
    
    test_results = [] # where we will be appending all our results
    alpha_list = (0.1, 0.05, 0.01) # the different alpha levels to test
    
    for i in range(4):
        for j in range(i + 1, 4):
            samp1, samp2 = quadrant_names[i], quadrant_names[j]
            t_stat, p_value = ttest_ind(quadrant_data[i], quadrant_data[j], equal_var=False)
            alpha_1, alpha_05, alpha_01 = (p_value < alpha_list[0],p_value < alpha_list[1], p_value < alpha_list[2])
            
            test_results.append((samp1, samp2, t_stat, p_value, alpha_1, alpha_05, alpha_01))
    
    return pd.DataFrame(test_results, columns=["quadrant_1", "quadrant_2", "t_stat", "p_value", "alpha 10%", "alpha 5%", "alpha 1%"])
            

In [70]:
test_quadrants(df).to_csv("tables/landmass.csv", index=False)
test_quadrants(df)

,quadrant_1,quadrant_2,t_stat,p_value,alpha 10%,alpha 5%,alpha 1%
0,NE,NW,-2.541272,0.011108,True,True,False
1,NE,SW,-0.919997,0.357687,False,False,False
2,NE,SE,1.629069,0.103324,False,False,False
3,NW,SW,1.378149,0.168233,False,False,False
4,NW,SE,2.870973,0.004130,True,True,True
5,SW,SE,1.300079,0.193734,False,False,False


## Strikes per hemisphere

In [53]:
def test_hemisphere(df):
    north = df[df.reclat > 0].mass
    south = df[df.reclat < 0].mass
    
    t_stat, p_value = ttest_ind(north, south, equal_var=False)
    alpha_list = (0.1, 0.05, 0.01) # the different alpha levels to test
    
    alpha_1, alpha_05, alpha_01 = (p_value < alpha_list[0],p_value < alpha_list[1], p_value < alpha_list[2])
    
    
#     plt.bar(["North", "South"], [north.sum(), south.sum()])
#     sns.distplot(north)
#     sns.distplot(south)
    return pd.DataFrame([(t_stat, p_value, alpha_1, alpha_05, alpha_01)], columns=["t_stat", "p_value", "alpha_1", "alpha_05", "alpha_01"])
    
test_hemisphere(df)

,t_stat,p_value,alpha_1,alpha_05,alpha_01
0,2.923305,0.00347,True,True,True


Based on the p value, we can conclude that the mass of meteorites that land in the northern hemisphere vs those that land in the south ar different. 

## Proportion test of found to fell

In [54]:
def test_fell_sightings_proportions(df, country1, country2):
    # Excerpt the two countries from the dataframe
    ctry1 = df[df.country == country1]
    ctry2 = df[df.country == country2]
    
    # get the total number of sightings/findings in a country
    ctry1_total = len(ctry1.fall)
    ctry2_total = len(ctry2.fall)
    
    #get the total number of sightings while falling in each country
    ctry1_sightings = len(ctry1[ctry1.fall == 'Fell'])
    ctry2_sightings = len(ctry2[ctry2.fall == 'Fell'])
    
    # run the proportion test
    z_stat, p_value = proportions_ztest([ctry1_sightings, ctry2_sightings], [ctry1_total, ctry2_total])
    
    
    return pd.DataFrame([(country1, country2, z_stat, p_value)], columns=("Country_1", "Country_2", "z_stat", "p_value"))

    
test_fell_sightings_proportions(df, "United States", "China")
    

,Country_1,Country_2,z_stat,p_value
0,United States,China,-16.022224,8.939509e-58


## Proportion Test of sightings by range of years

In [75]:
# df[df.fall == "Fell"].sort_values("year")

In [88]:
def test_fall_sightings_dates(df, start_date, split_date, end_date):
    
    range_1 = df[(df.year >= start_date) & (df.year < split_date)]
    range_2 = df[(df.year >= split_date) & (df.year <= end_date )]
    
    range_1_name = f"{start_date} - {split_date - 1}"
    range_2_name = f"{split_date} - {end_date}"
    
    range_1_total = len(range_1.fall)
    range_2_total = len(range_2.fall)
    
    range_1_sightings = len(range_1[range_1.fall == "Fell"])
    range_2_sightings = len(range_2[range_2.fall == "Fell"])
    
    z_stat, p_value = proportions_ztest((range_1_sightings, range_2_sightings), (range_1_total, range_2_total))
                                        
    return pd.DataFrame(
        [(range_1_name, range_2_name, z_stat, p_value)],
        columns=("Date Range 1", "Date Range 2", "z-stat", "p-value")
    )

In [89]:
test_fall_sightings_dates(df, 1950, 1990, 2010).to_csv('tables/proportion_fell_by_year.csv', index=False)
test_fall_sightings_dates(df, 1950, 1990, 2010)

,Date Range 1,Date Range 2,z-stat,p-value
0,1950 - 1989,1990 - 2010,9.6358,5.645074e-22


In [60]:
earth_strikes = pd.read_csv("CSV_MASTERS/GeoEarth_with_num_strikes")
earth_strikes.loc[121, "subregion"] = "Antarctica"
earth_strikes.loc[121, "region_wb"] = "Antarctica"

In [81]:
region = list(earth_strikes.region_wb.unique())

samples = []

for r in region:
    samples.append(earth_strikes[earth_strikes.region_wb == r].num_strikes)


    
test_results = []
alpha_list = (0.1, 0.05, 0.01) # the different alpha levels to test
for i in range(len(samples)):
    for j in range(i + 1, len(samples)):
        samp1, samp2 = samples[i], samples[j]
        t_stat, p_value = ttest_ind(samp1, samp2, equal_var=False)
        alpha_1, alpha_05, alpha_01 = (p_value < alpha_list[0],p_value < alpha_list[1], p_value < alpha_list[2])
        
        test_results.append((region[i], region[j], t_stat, p_value, alpha_1, alpha_05, alpha_01))
        
region_strikes = pd.DataFrame(test_results, columns=["Region 1", "Region 2", "t-stat", "p-value", "alpha 10%", "alpha 5%", "alpha 1%"])

region_strikes.dropna(inplace=True)

/home/qdizon/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3194: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/home/qdizon/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [84]:
region_strikes.to_csv("tables/strikes_per_region.csv", index=False)
region_strikes

,Region 1,Region 2,t-stat,p-value,alpha 10%,alpha 5%,alpha 1%
0,Latin America & Caribbean,North America,-0.886507,0.534286,False,False,False
1,Latin America & Caribbean,Europe & Central Asia,1.238742,0.234426,False,False,False
2,Latin America & Caribbean,Middle East & North Africa,-0.960486,0.347235,False,False,False
3,Latin America & Caribbean,South Asia,1.056172,0.306036,False,False,False
4,Latin America & Caribbean,East Asia & Pacific,0.791141,0.438777,False,False,False
5,Latin America & Caribbean,Sub-Saharan Africa,1.220304,0.240944,False,False,False
8,North America,Europe & Central Asia,1.055162,0.482911,False,False,False
9,North America,Middle East & North Africa,0.594272,0.648478,False,False,False
10,North America,South Asia,1.034743,0.488864,False,False,False
11,North America,East Asia & Pacific,1.002038,0.498793,False,False,False
